In [1]:
import os

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

import re
import cv2
import time
import numpy as np
import pandas as pd


import torch
import torch.optim as optim
import torchvision.transforms as T
from torchvision.utils import make_grid 
from torch.utils.data import DataLoader, Dataset

from retinanet.csv_eval import evaluate 
from retinanet import model
from retinanet.dataloader import collater, Resizer, Augmenter, Normalizer, UnNormalizer,CSVDataset


In [2]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [3]:
class RT_Dataset(Dataset):
    def __init__(self,dataframe,image_dir,transforms=None):
        super().__init__()
        self.image_ids=dataframe['image_number'].unique()
        self.df=dataframe
        self.image_dir=image_dir
        self.transforms=transforms
        
    def __getitem__(self,index:int):
        image_id=self.image_ids[index]
        records=self.df[self.df['image_number']==image_id]
        
        image=cv2.imread(f'{self.image_dir}/{records["dataset"].values[0]}/{records["image_name"].values[0]}',cv2.IMREAD_COLOR)
        image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB).astype(np.float32)
        image/=255.0
        
        bndbox=np.array(eval(records['bndbox'].values[0]))
        labels=np.array(eval(records['labels'].values[0]))
        boxes=np.zeros((bndbox.shape[0],5))
        
        if bndbox.size>0:
            boxes[:,0:4]=bndbox
            boxes[:,4]=labels
        
        sample={'img':image, 'annot':boxes}
        
        if self.transforms:
            sample=self.transforms(sample)
            
        return sample
        
    def __len__(self) -> int:
        return self.image_ids.shape[0]

In [4]:
test_df=pd.read_csv('../../data/annotations/test_total.csv')
image_dir='/home/irteam/junghye-dcloud-dir/pathfinder/data_contrast/after/Image'
test_dataset=RT_Dataset(test_df,image_dir,transforms=T.Compose([Normalizer(),Resizer()]))

test_data_loader=DataLoader(
    test_dataset,
    batch_size=2,
    shuffle=False,
    num_workers=4,
    collate_fn=collater
)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
torch.cuda.empty_cache()


In [5]:
retinanet = torch.load("/home/irteam/junghye-dcloud-dir/pathfinder/pathfinder_ai/retinanet_/models/retinanet_AFTER_150.pt", map_location=device)

# 모델을 평가 모드로 전환합니다.
retinanet.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [6]:
average_precisions = evaluate(test_dataset, retinanet, iou_threshold=0.000000001, score_threshold=0.000000001, max_detections=100000, num_classes=3,save_path='./')

scores:[],labels:[],boxes:[]
scores:[],labels:[],boxes:[]
scores:[],labels:[],boxes:[]
scores:[],labels:[],boxes:[]
scores:[],labels:[],boxes:[]
scores:[],labels:[],boxes:[]
scores:[],labels:[],boxes:[]
scores:[],labels:[],boxes:[]
scores:[],labels:[],boxes:[]
scores:[],labels:[],boxes:[]
scores:[],labels:[],boxes:[]
scores:[],labels:[],boxes:[]
scores:[],labels:[],boxes:[]
scores:[],labels:[],boxes:[]
scores:[],labels:[],boxes:[]
scores:[],labels:[],boxes:[]
scores:[],labels:[],boxes:[]
scores:[],labels:[],boxes:[]
scores:[],labels:[],boxes:[]
scores:[],labels:[],boxes:[]
scores:[],labels:[],boxes:[]
scores:[],labels:[],boxes:[]
scores:[],labels:[],boxes:[]
scores:[],labels:[],boxes:[]
scores:[],labels:[],boxes:[]
scores:[],labels:[],boxes:[]
scores:[],labels:[],boxes:[]
scores:[],labels:[],boxes:[]
scores:[],labels:[],boxes:[]
scores:[],labels:[],boxes:[]
scores:[],labels:[],boxes:[]
scores:[],labels:[],boxes:[]
scores:[],labels:[],boxes:[]
scores:[],labels:[],boxes:[]
scores:[],labe

IndexError: index -1 is out of bounds for axis 0 with size 0